In [402]:
import re

#Functions

In [403]:
def replace_single_quotes(text):
    # Use regular expressions to find and replace ' after alphabetic characters
    modified_text = re.sub(r'([a-zA-Z])\'', r'~\1', text)
    modified_text = modified_text.replace('+','|')
    modified_text = modified_text.replace('*','&')
    return modified_text


In [404]:
def extract_variables(expression):
    # Use regular expressions to find variable names
    variable_pattern = r'\b[a-zA-Z_][a-zA-Z0-9_]*\b'
    variables = re.findall(variable_pattern, expression)

    # Remove duplicates by converting to a set and then back to a list
    unique_variables = list(set(variables))

    # Sort the variables alphabetically
    unique_variables.sort()

    return unique_variables


In [405]:
def replace_single_quotes2(text):
    # Use regular expressions to find and replace ' after alphabetic characters
    modified_text = text.replace('~',' not ')
    modified_text = modified_text.replace('|',' or ')
    modified_text = modified_text.replace('&',' and ')
    return modified_text


In [406]:
def get_bits_with_fixed_length(number, lenght):
    # Get the binary representation of the number
    bits = []
    while number:
        bits.append(number & 1)
        number >>= 1
    bits.reverse()

    # Calculate the number of leading zeros to add
    leading_zeros = [0] * (lenght - len(bits))

    # Combine leading zeros and the actual bits
    result = leading_zeros + bits

    return result

In [407]:
def get_truth_table(user_input):
  var_list = extract_variables(replace_single_quotes(user_input))
  table_list=[]
  for j in range(2**(len(var_list))):
    number_bits= get_bits_with_fixed_length(j, len(var_list))
    temp = replace_single_quotes(user_input)
    for i in range(len(var_list)):
      temp = temp.replace(var_list[i], str(number_bits[i]))
    temp = temp.replace("~1", "0").replace("~0","1")
    table_list.append(eval(temp))
  return table_list


In [408]:
def SOP(user_input):
  SOP_Index=[]
  truth_table = get_truth_table(user_input)
  for i in range(len(truth_table)):
    if truth_table[i]:
      SOP_Index.append(i);
  sigma_sign = '\u03A3'
  # print(" Canonical SOP = "+sigma_sign+str(SOP_Index).replace('[','(').replace(']',')'))
  return SOP_Index

In [409]:
def POS(user_input):
  POS_Index=[]
  truth_table = get_truth_table(user_input)
  for i in range(len(truth_table)):
    if truth_table[i]==0:
      POS_Index.append(i);
  #print(" Canonical POS = "+"PI"+str(POS_Index).replace('[','(').replace(']',')'))
  return POS_Index

In [410]:
def INVERSE_SOP(user_input):
  INVERSE_SOP_Index=[]
  truth_table = get_truth_table(user_input)
  for i in range(len(truth_table)):
    if truth_table[i]==0:
      INVERSE_SOP_Index.append(i);
  return INVERSE_SOP_Index

In [411]:
def INVERSE_POS(user_input):
  INVERSE_POS_Index=[]
  truth_table = get_truth_table(user_input)
  for i in range(len(truth_table)):
    if truth_table[i]:
      INVERSE_POS_Index.append(i);
  return INVERSE_POS_Index


In [412]:
from IPython.core.display import deepcopy

def mul(x,y):
    res = []
    for i in x:
        if i+"'" in y or (len(i)==2 and i[0] in y):
            return []
        else:
            res.append(i)
    for i in y:
        if i not in res:
            res.append(i)
    return res

def multiply(x,y):
    res = []
    for i in x:
        for j in y:
            tmp = mul(i,j)
            res.append(tmp) if len(tmp) != 0 else None
    return res

def EPI_finging(x):
    res = []
    for i in x:
        if len(x[i]) == 1:
            res.append(x[i][0]) if x[i][0] not in res else None
    return res

def get_var(x):
    var_list = []
    for i in range(len(x)):
        if x[i] == '0':
            var_list.append(chr(i+65)+"'")
        elif x[i] == '1':
            var_list.append(chr(i+65))
    return var_list

def flatten(x):
    flattened_items = []
    for i in x:
        flattened_items.extend(x[i])
    return flattened_items

def get_minterms(a):
    gaps = a.count('-')
    if gaps == 0:
        return [str(int(a,2))]
    x = [bin(i)[2:].zfill(gaps) for i in range(pow(2,gaps))]
    temp = []
    for i in range(pow(2,gaps)):
        temp2,ind = a[:],-1
        for j in x[0]:
            if ind != -1:
                ind = ind+temp2[ind+1:].find('-')+1
            else:
                ind = temp2[ind+1:].find('-')
            temp2 = temp2[:ind]+j+temp2[ind+1:]
        temp.append(str(int(temp2,2)))
        x.pop(0)
    return temp

def compare(a,b):
    c = 0
    for i in range(len(a)):
        if a[i] != b[i]:
            mismatch_index = i
            c += 1
            if c>1:
                return (False,None)
    return (True,mismatch_index)

def remove_Item(_chart,terms):
    for i in terms:
        for j in get_minterms(i):
            try:
                del _chart[j]
            except KeyError:
                pass

def Quine_McCluskey(user_input):

  minterms = SOP(user_input);
  size = len(extract_variables(user_input));
  primeImplicant_List=[]
  Lists=[]
  for i in range(size+1):
    Lists.append([])
  for minterm in minterms:
          Lists[bin(minterm).count('1')].append(bin(minterm)[2:].zfill(size))
  while True:
    temp=[];
    temp_Lists=deepcopy(Lists);
    Lists=[]
    for i in range(size+1):
      Lists.append([])
    m=0;
    marked=[];
    local_unmarked=[];
    should_stop = True;
    for i in range(size+1):
      temp.append(i)
    for i in range(len(temp)-1):
            for j in temp_Lists[temp[i]]:
                for k in temp_Lists[temp[i+1]]:
                    res = compare(j,k)
                    if res[0]:
                        try:
                            Lists[m].append(j[:res[1]]+'-'+j[res[1]+1:]) if j[:res[1]]+'-'+j[res[1]+1:] not in Lists[m] else None
                        except KeyError:
                            Lists[m] = [j[:res[1]]+'-'+j[res[1]+1:]]
                        should_stop = False
                        if j not in marked:
                          marked.append(j)
                        if k not in marked:
                          marked.append(k)
            m += 1
    local_unmarked=[]
    for item in temp_Lists:
      for item2 in item:
        if item2 not in marked:
          local_unmarked.append(item)
    for item in local_unmarked:
      if item not in primeImplicant_List:
        primeImplicant_List.append(item)
    if should_stop:
        break

  primeImplicant = []
  for item in primeImplicant_List:
    for item2 in item:
      primeImplicant.append(item2)
  sz = len(str(minterms[-1]))
  chart = {}
  for i in primeImplicant:
      merged_minterms,y = get_minterms(i),0
      for j in merged_minterms:
          x = minterms.index(int(j))*(sz+1)
          y = x+sz
          try:
              chart[j].append(i) if i not in chart[j] else None
          except KeyError:
              chart[j] = [i]
  EPI = EPI_finging(chart)
  remove_Item(chart,EPI)
  if(len(chart) == 0):
      ESS_PI = [get_var(i) for i in EPI]
  else:
      P = [[get_var(j) for j in chart[i]] for i in chart]
      while len(P)>1:
          P[1] = multiply(P[0],P[1])
          P.pop(0)
      ESS_PI = [min(P[0],key=len)]
      ESS_PI.extend(get_var(i) for i in EPI)
  return primeImplicant, ESS_PI

In [413]:
def minimized_SOP(user_input):
  sop_num=len(SOP(user_input))
  minimum_num=len(Quine_McCluskey(user_input)[1])
  print("number of saved literals =",minimum_num,"number of the canonical version=", sop_num)

In [414]:
def minimized_POS(user_input):
  Pos_num=len(POS(user_input))
  maxterm_num=len(Quine_McCluskey("~("+user_input+")")[1])
  print("number of saved literals =",maxterm_num,"number of the canonical version=", Pos_num)

In [415]:
def findPrimeImplicants(user_input):
  minimum_num=len(Quine_McCluskey(user_input)[0])
  print("number of Prime implicants are: ",minimum_num)

In [416]:
def findEssPrimeImplicants(user_input):
  minimum_num=len(Quine_McCluskey(user_input)[1])
  print("number of Essential Prime implicants are: ",minimum_num)

In [417]:
def onSetMinterms(user_input):
  print("the number of Onset_minterms =",sum(get_truth_table(user_input)))

In [418]:
def OnSetMaxterms(user_input):
  print("the number of Onset_maxterms =", sum([1 if x == 0 else 0 for x in get_truth_table(user_input)]))

In [427]:
def numberOfActiveInputs(user_input):
    temp = Quine_McCluskey(user_input)[1]
    temp2= 0
    for item in temp:
      for item2 in item:
        temp2 = temp2+1
    return temp2

In [447]:
def getEssPrimeImplicantExpression(user_input):
  temp = Quine_McCluskey(user_input)[1]
  a=""
  for item in temp:
    if len(item)==1:
      if a == "":
        a=a+str(item[0])
      else:
        a=a+"+"+str(item[0])
    else:
      for i in range(len(item)):
        if i==0:
          if a=="":
            a=a+str(item[i])
          else:
            a=a+"+"+str(item[i])
        else:
          a = a + "&" + str(item[i])
  return print(a)


In [419]:
def AIG():
  class AIGNode:
      def __init__(self, node_type, inputs):
          self.node_type = node_type
          self.inputs = inputs

  def aig_to_string(node_map, node_id):
      node = node_map[node_id]

      if node.node_type == "Input":
          return node.inputs
      elif node.node_type == "Inverter":
          input_node = node_map[node.inputs[0]]
          input_str = aig_to_string(node_map, node.inputs[0])
          return f"(~{input_str})"
      elif node.node_type == "AND":
          input1_str = aig_to_string(node_map, node.inputs[0])
          input2_str = aig_to_string(node_map, node.inputs[1])
          return f"({input1_str} * {input2_str})"
      else:
          return ""
  # Ask the user for the node_map and output_node_id
  node_map = {}
  while True:
      try:
          node_id = int(input("Enter node ID (or -1 to stop): "))
          if node_id == -1:
              break
          node_type = input("Enter node type (Input, Inverter, AND): ")
          if node_type == "Input":
              input_name = input("Enter input name: ")
              node = AIGNode(node_type, input_name)
          else:
              input_ids = input("Enter input node IDs (comma-separated): ").split(",")
              input_ids = [int(id) for id in input_ids]
              node = AIGNode(node_type, input_ids)
          node_map[node_id] = node
      except ValueError:
          print("Invalid input. Please enter a valid node ID and input values.")

  output_node_id = int(input("Enter the output node ID: "))


  expression = aig_to_string(node_map, output_node_id)
  return expression


#Run

In [448]:
def Run():
  print("Welcome to hamed's Program:\n")
  input_format = input("How do you want to give me the circuit? :\n 1.Boolean algebraic function \n 2. AIG \n")
  if input_format == '1':
    user_input = input("Enter a function: ")
  elif input_format == '2':
    user_input = AIG()
  else:
    print("please enter 1 or 2 for input format")
  while True:
    input_function = input("\n\n\n commands: \n1. Return the design as a canonical SOP\n2. Return the design as a canonical POS\n3. Return the design INVERSE as a canonical SOP\n4. Return the design INVERSE as a canonical POS\n5. Return a minimized number of literals representation in SOP\n6. Return a minimized number of literals representation in POS\n7. Report the number of Prime Implicants\n8. Report the number of Essential Prime Implicants\n9. Report the number of ON-Set minterms\n10. Report the number of ON-Set maxterms\n11. number of inputs that affects output:  \n12.get expression using Essential Prime \n13.exit\n")
    match input_function :
      case '1':
        print("\n\n\n")
        SOP_Index = SOP(user_input)
        sigma_sign = '\u03A3'
        print(" Canonical SOP = "+sigma_sign+str(SOP_Index).replace('[','(').replace(']',')'));

      case '2':
        print("\n\n\n")
        POS_Index = POS(user_input)
        print(" Canonical POS = "+"PI"+str(POS_Index).replace('[','(').replace(']',')'))
      case '3':
        print("\n\n\n")
        INVERSE_SOP_Index = INVERSE_SOP(user_input)
        sigma_sign = '\u03A3'
        print(" Inverse Canonical SOP = "+sigma_sign+str(INVERSE_SOP_Index).replace('[','(').replace(']',')'))
      case '4':
        print("\n\n\n")
        INVERSE_POS_Index = INVERSE_POS(user_input)
        print(" Inverse Canonical POS = "+"PI"+str(INVERSE_POS_Index).replace('[','(').replace(']',')'))
      case '5':
          print("\n\n\n")
          minimized_SOP(user_input);
      case '6':
          print("\n\n\n")
          minimized_POS(user_input);
      case '7':
          print("\n\n\n")
          findPrimeImplicants(user_input);
      case '8':
          print("\n\n\n")
          findEssPrimeImplicants(user_input);
      case '9':
          print("\n\n\n")
          onSetMinterms(user_input);
      case '10':
          print("\n\n\n")
          OnSetMaxterms(user_input);
      case '11':
          print("\n\n\n")
          print("number of inputs that affect the output: ",numberOfActiveInputs(user_input))
      case '12':
          print("\n\n\n")
          getEssPrimeImplicantExpression(user_input)
      case default:
          break



In [449]:
Run()

Welcome to hamed's Program:

How do you want to give me the circuit? :
 1.Boolean algebraic function 
 2. AIG 
1
Enter a function: a+b*~a+c*a



 commands: 
1. Return the design as a canonical SOP
2. Return the design as a canonical POS
3. Return the design INVERSE as a canonical SOP
4. Return the design INVERSE as a canonical POS
5. Return a minimized number of literals representation in SOP
6. Return a minimized number of literals representation in POS
7. Report the number of Prime Implicants
8. Report the number of Essential Prime Implicants
9. Report the number of ON-Set minterms
10. Report the number of ON-Set maxterms
 
 11. number of inputs that affects output:  
12.get expression using Essential Prime 
13.exit
11




number of inputs that affect the output:  2



 commands: 
1. Return the design as a canonical SOP
2. Return the design as a canonical POS
3. Return the design INVERSE as a canonical SOP
4. Return the design INVERSE as a canonical POS
5. Return a minimized number o

KeyboardInterrupt: ignored